<a href="https://colab.research.google.com/github/naiaraAM/ML_project_UT/blob/main/Project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Set up enviroment


# Download data


In [7]:
import pandas as pd
import numpy as np

In [6]:
!unzip -q playground-series-s4e6.zip -d data
!rm playground-series-s4e6.zip

unzip:  cannot find or open playground-series-s4e6.zip, playground-series-s4e6.zip.zip or playground-series-s4e6.zip.ZIP.
rm: cannot remove 'playground-series-s4e6.zip': No such file or directory


In [2]:
from sklearn.model_selection import train_test_split
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

print(f"The train data has {train_data.shape[0]} rows and {train_data.shape[1]} columns")
print(f"The test data has {test_data.shape[0]} rows and {test_data.shape[1]} columns")

The train data has 76518 rows and 38 columns
The test data has 51012 rows and 37 columns


<font color='red'>Do your code in the cell bellow your name, so we don't have merge issues. We can merge on friday's meeting</font>


## Naiara


In [20]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import gc
import time
import matplotlib

train_data = pd.read_csv('data/train.csv')

features = train_data.columns[:-1]

max_unique_values = 100  # Different possible values for feature
features = [f for f in features if train_data[f].nunique() <= max_unique_values]

output_dir = 'plots'
os.makedirs(output_dir, exist_ok=True)


matplotlib.use('Agg')

batch_size = 5
for i in range(0, len(features), batch_size):
    batch_features = features[i:i + batch_size]
    
    for feature in batch_features:
        try:
            print(f"Processing feature: {feature}")
            percentage = train_data.groupby(feature)['Target'].value_counts(normalize=True).unstack() * 100
            
            # Plot and save
            ax = percentage.plot(kind='bar', stacked=True, colormap='viridis', legend=False)
            plt.xlabel(feature)
            plt.ylabel('Percentage')
            plt.title(f'Percentage of Each Target Value per {feature} Value')
            handles, labels = ax.get_legend_handles_labels()
            plt.legend(handles, labels, title='Target')

            # Save plot
            plot_filename = os.path.join(output_dir, f'{feature}_percentage_plot.png')
            plt.savefig(plot_filename)
            plt.close()
            #print(f"Saved plot for feature: {feature}")
            
            time.sleep(0.5)

        except Exception as e:
            print(f"Error processing feature {feature}: {e}")

        gc.collect()  # Clear memory
    
    #print(f"Batch {i // batch_size + 1} completed.")

Processing feature: Marital status
Processing feature: Application mode
Processing feature: Application order
Processing feature: Course
Processing feature: Daytime/evening attendance
Error processing feature Daytime/evening attendance: [Errno 2] No such file or directory: '/gpfs/helios/home/naiara/machine_learning/ML_project_UT/plots/Daytime/evening attendance_percentage_plot.png'
Processing feature: Previous qualification
Processing feature: Nacionality
Processing feature: Mother's qualification
Processing feature: Father's qualification
Processing feature: Mother's occupation
Processing feature: Father's occupation
Processing feature: Displaced
Processing feature: Educational special needs
Processing feature: Debtor
Processing feature: Tuition fees up to date
Processing feature: Gender
Processing feature: Scholarship holder
Processing feature: Age at enrollment
Processing feature: International
Processing feature: Curricular units 1st sem (credited)
Processing feature: Curricular un

In [12]:
def submitModel(model, X_test):
    X_test_scaled = scaler.transform(X_test)
    Y_tilda = model.predict(X_test_scaled)
    submission = pd.DataFrame(
        {'id': X_test["id"], 'Target': Y_tilda},
        columns = ['id', 'Target'])
    submission.to_csv('submission.csv', index = False)

In [28]:
submitModel(best_model, test_data) # 0.82420

In [12]:
import pickle

def save_model(models, filename: str):
    for index, model in enumerate(models):
        filename_aux = filename + f'_{index}.pkl'
        with open(filename_aux, 'wb') as file:  
            pickle.dump(model, file)
            print(f"Saved with the name {filename_aux}")

# TODO: I need to fix this
def load_model(filename: str):
    with open(filename, 'rb') as file:  
        return pickle.load(file)

Non-equal performance on public-private score, public is always better

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

X = train_data.iloc[:, :-1]
y = train_data.iloc[:, -1]

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(y.value_counts())

X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

gbt_model = GradientBoostingClassifier(n_estimators=350, learning_rate=0.03, max_depth=5)

gbt_model.fit(X_train, y_train)

y_pred_val = gbt_model.predict(X_val)

accuracy_val = accuracy_score(y_val, y_pred_val)
print(f"Validation Accuracy: {accuracy_val:.4f}")

print(classification_report(y_val, y_pred_val))

Target
Graduate    36282
Dropout     25296
Enrolled    14940
Name: count, dtype: int64
Validation Accuracy: 0.8287
              precision    recall  f1-score   support

           0       0.90      0.83      0.86      5059
           1       0.65      0.60      0.62      2988
           2       0.85      0.92      0.89      7257

    accuracy                           0.83     15304
   macro avg       0.80      0.78      0.79     15304
weighted avg       0.83      0.83      0.83     15304



In [36]:
filename = "XXXXX_gbt_model.pkl"
save_model(filename)
#model = load_model(filename)

y_test = model.predict(X_test_pca)
y_test = label_encoder.inverse_transform(y_test)

submission = pd.DataFrame(
    {'id': test_data["id"], 'Target': y_test},
    columns=['id', 'Target']
)
submission.to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'")

Saved with the name XXXXX_gbt_model.pkl
Submission file saved as 'submission.csv'


In [21]:
data_depth = {
    'n_estimators': [200, 200, 200, 200, 200, 200, 200, 200, 200],
    'learning_rate': [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'accuracy': [0.8149, 0.8225, 0.8268, 0.8265, 0.8218, 0.8317, 0.8295, 0.8308, 0.8300],
    'private score': [0.82227, 0.82737, 0.83055, 0.83124, 0.83094, 0.83119, 0.82378, 0.82337, 0.82276],
    'public score': [0.81915, 0.82513, 0.82915, 0.83022, 0.83199, 0.83120, 0.83150, 0.82111, 0.81866]
}

df_depth = pd.DataFrame(data_depth)

df_styled = df_depth.style.background_gradient(subset=["accuracy", "public score", "private score"], cmap="RdYlGn", vmin=0.8148, vmax=0.84)

df_styled

,n_estimators,learning_rate,max_depth,accuracy,private score,public score
0,200,0.050000,1,0.814900,0.822270,0.819150
1,200,0.050000,2,0.822500,0.827370,0.825130
2,200,0.050000,3,0.826800,0.830550,0.829150
3,200,0.050000,4,0.826500,0.831240,0.830220
4,200,0.050000,5,0.821800,0.830940,0.831990
5,200,0.050000,6,0.831700,0.831190,0.831200
6,200,0.050000,7,0.829500,0.823780,0.831500
7,200,0.050000,8,0.830800,0.823370,0.821110
8,200,0.050000,9,0.830000,0.822760,0.818660


In [22]:
data_learning_rate = {
    'n_estimators': [200, 200, 200, 200, 200, 200],
    'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06],
    'max_depth': [6, 6, 6, 6, 6, 6],
    'accuracy': [0.8228, 0.8291, 0.8304, 0.8304, 0.8306, 0.8298],
    'private score': [0.82599, 0.82969, 0.83126, 0.83094, 0.82967, 0.82435],
    'public score': [0.82464, 0.82826, 0.82856, 0.83071, 0.83179, 0.82415]
}

df_learning_rate = pd.DataFrame(data_learning_rate)

df_styled = df_learning_rate.style.background_gradient(subset=["accuracy", "public score", "private score"], cmap="RdYlGn", vmin=0.821, vmax=0.84)

df_styled

,n_estimators,learning_rate,max_depth,accuracy,private score,public score
0,200,0.010000,6,0.822800,0.825990,0.824640
1,200,0.020000,6,0.829100,0.829690,0.828260
2,200,0.030000,6,0.830400,0.831260,0.828560
3,200,0.040000,6,0.830400,0.830940,0.830710
4,200,0.050000,6,0.830600,0.829670,0.831790
5,200,0.060000,6,0.829800,0.824350,0.824150


In [23]:
data_estimators = {
    'n_estimators': [50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300],
    'learning_rate': [0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03],
    'max_depth': [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
    'accuracy': [0.8200, 0.8248, 0.8276, 0.8284, 0.8292, 0.8296, 0.8304, 0.8306, 0.8314, 0.8311, 0.8314],
    'private score': [0.82349, 0.82683, 0.82854, 0.82967, 0.83092, 0.83109, 0.83146, 0.83002, 0.83033, 0.83075, 0.83053],
    'public score': [0.82268, 0.82503, 0.82738, 0.82866, 0.82866, 0.82875, 0.82787, 0.83022, 0.83091, 0.83052, 0.83120]
}

df_estimators = pd.DataFrame(data_estimators)

df_styled = df_estimators.style.background_gradient(subset=["accuracy", "public score", "private score"], cmap="RdYlGn", vmin=0.8148, vmax=0.84)

df_styled

,n_estimators,learning_rate,max_depth,accuracy,private score,public score
0,50,0.030000,6,0.820000,0.823490,0.822680
1,75,0.030000,6,0.824800,0.826830,0.825030
2,100,0.030000,6,0.827600,0.828540,0.827380
3,125,0.030000,6,0.828400,0.829670,0.828660
4,150,0.030000,6,0.829200,0.830920,0.828660
5,175,0.030000,6,0.829600,0.831090,0.828750
6,200,0.030000,6,0.830400,0.831460,0.827870
7,225,0.030000,6,0.830600,0.830020,0.830220
8,250,0.030000,6,0.831400,0.830330,0.830910
9,275,0.030000,6,0.831100,0.830750,0.830520


In [24]:
data_estimators = {
    'n_estimators': [300, 350],
    'learning_rate': [0.03, 0.03],
    'max_depth': [5, 5],
    'private score': [0.83080, 0.83082],
    'public score': [0.83189, 0.83238]
}

df_estimators = pd.DataFrame(data_estimators)

df_styled = df_estimators.style.background_gradient(subset=["public score", "private score"], cmap="RdYlGn", vmin=0.8148, vmax=0.84)

df_styled

,n_estimators,learning_rate,max_depth,private score,public score
0,300,0.030000,5,0.830800,0.831890
1,350,0.030000,5,0.830820,0.832380


This has similar performance on private-public, but the other approach is better.

What this approach does. Separated the tags into three groups:
* 0 and 1 (after label encoder)
* 0 and 2 (after label encoder)
* 1 and 2 (after label encoder)
  
Train the three models separately. Then combines
* if prediction from `model_01` and `model_02`, then prediction is `0`
* if prediction from `model_01` and `model_12`, then prediction is `1`
* if prediction from `model_02` and `model_12`, then prediction is `2`
* else majority voting

I realised that the models struggle to predict for class `1`. If I change the `GradientBoostingClassifier` to `HistGradientBoostingClassifier` it performs a little b

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

X = train_data.iloc[:, :-1]
y = train_data.iloc[:, -1]

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

X_train_01 = X_train[(y_train == 0) | (y_train == 1)]
y_train_01 = y_train[(y_train == 0) | (y_train == 1)]
X_val_01 = X_val[(y_val == 0) | (y_val == 1)]
y_val_01 = y_val[(y_val == 0) | (y_val == 1)]

X_train_02 = X_train[(y_train == 0) | (y_train == 2)]
y_train_02 = y_train[(y_train == 0) | (y_train == 2)]
X_val_02 = X_val[(y_val == 0) | (y_val == 2)]
y_val_02 = y_val[(y_val == 0) | (y_val == 2)]

X_train_12 = X_train[(y_train == 1) | (y_train == 2)]
y_train_12 = y_train[(y_train == 1) | (y_train == 2)]
X_val_12 = X_val[(y_val == 1) | (y_val == 2)]
y_val_12 = y_val[(y_val == 1) | (y_val == 2)]

model_01 = GradientBoostingClassifier(n_estimators=350, learning_rate=0.03, max_depth=5)
model_01.fit(X_train_01, y_train_01)
y_pred_val_01 = model_01.predict(X_val_01)
accuracy_val = accuracy_score(y_val_01, y_pred_val_01)
print(f"Validation Accuracy for model 01: {accuracy_val:.4f}")
print(classification_report(y_val_01, y_pred_val_01))

model_02 = GradientBoostingClassifier(n_estimators=350, learning_rate=0.03, max_depth=5)
model_02.fit(X_train_02, y_train_02)
y_pred_val_02 = model_02.predict(X_val_02)
accuracy_val = accuracy_score(y_val_02, y_pred_val_02)
print(f"Validation Accuracy for model 02: {accuracy_val:.4f}")
print(classification_report(y_val_02, y_pred_val_02))

model_12 = GradientBoostingClassifier(n_estimators=350, learning_rate=0.03, max_depth=5)
model_12.fit(X_train_12, y_train_12)
y_pred_val_12 = model_12.predict(X_val_12)
accuracy_val = accuracy_score(y_val_12, y_pred_val_12)
print(f"Validation Accuracy for model 12: {accuracy_val:.4f}")
print(classification_report(y_val_12, y_pred_val_12))

Validation Accuracy for model 01: 0.8525
              precision    recall  f1-score   support

           0       0.92      0.84      0.88      5059
           1       0.76      0.87      0.81      2988

    accuracy                           0.85      8047
   macro avg       0.84      0.86      0.85      8047
weighted avg       0.86      0.85      0.85      8047

Validation Accuracy for model 02: 0.9445
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      5059
           2       0.94      0.97      0.95      7257

    accuracy                           0.94     12316
   macro avg       0.95      0.94      0.94     12316
weighted avg       0.94      0.94      0.94     12316

Validation Accuracy for model 12: 0.8682
              precision    recall  f1-score   support

           1       0.81      0.71      0.76      2988
           2       0.89      0.93      0.91      7257

    accuracy                           0.87     10245
   ma

In [26]:
from sklearn.metrics import classification_report, accuracy_score
def predict(X_data):
    final_predictions = []
    
    for X_pred_data in X_data:
        # Predictions for every model
        predict_01 = model_01.predict([X_pred_data])[0]
        predict_02 = model_02.predict([X_pred_data])[0]
        predict_12 = model_12.predict([X_pred_data])[0]
        
        if predict_01 == predict_02:  # High chance of being class 0
            prediction = 0
        elif predict_01 == predict_12:  # High change of being class 1
            prediction = 1
        elif predict_02 == predict_12:  # High chance of being class 2
            prediction = 2
        else:  # mayority voto
            votes = [predict_01, predict_02, predict_12]
            prediction = max(set(votes), key=votes.count)
        final_predictions.append(prediction)
    return np.array(final_predictions)
    
final_predictions = predict(X_val)

accuracy_val_ensemble = accuracy_score(y_val, final_predictions)
print(f"Validation Accuracy for Ensemble Model: {accuracy_val_ensemble:.4f}")

print(classification_report(y_val, final_predictions, zero_division=1))

Validation Accuracy for Ensemble Model: 0.8280
              precision    recall  f1-score   support

           0       0.90      0.83      0.86      5059
           1       0.64      0.60      0.62      2988
           2       0.85      0.92      0.89      7257

    accuracy                           0.83     15304
   macro avg       0.80      0.78      0.79     15304
weighted avg       0.83      0.83      0.83     15304



In [27]:
models = [model_01, model_02, model_12]
filename = "XXXXX_gbc_ensemble_model"
save_model(models, filename)
#model = load_model(filename)
test_data = pd.read_csv('data/test.csv')
scaler = StandardScaler()
X_scaled = scaler.fit_transform(test_data)
final_predictions = predict(X_scaled)
y_test = label_encoder.inverse_transform(final_predictions)

submission = pd.DataFrame(
    {'id': test_data["id"], 'Target': y_test},
    columns=['id', 'Target']
)
submission.to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'")

Saved with the name XXXXX_gbc_ensemble_model_0.pkl
Saved with the name XXXXX_gbc_ensemble_model_1.pkl
Saved with the name XXXXX_gbc_ensemble_model_2.pkl
Submission file saved as 'submission.csv'


## Juan

In [11]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np

In [7]:
train_data.describe(include='all')

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
count,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,...,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Graduate
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36282
mean,38258.500000,1.111934,16.054419,1.644410,9001.286377,0.915314,3.658760,132.378766,1.226600,19.837633,...,0.137053,5.933414,7.234468,4.007201,9.626085,0.062443,11.520340,1.228218,-0.080921,NaN
std,22088.988286,0.441669,16.682337,1.229645,1803.438531,0.278416,8.623774,10.995328,3.392183,15.399456,...,0.933830,1.627182,3.503040,2.772956,5.546035,0.462107,2.653375,1.398816,2.251382,NaN
min,0.000000,1.000000,1.000000,0.000000,33.000000,0.000000,1.000000,95.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000,NaN
25%,19129.250000,1.000000,1.000000,1.000000,9119.000000,1.000000,1.000000,125.000000,1.000000,1.000000,...,0.000000,5.000000,6.000000,1.000000,10.000000,0.000000,9.400000,0.300000,-1.700000,NaN
50%,38258.500000,1.000000,17.000000,1.000000,9254.000000,1.000000,1.000000,133.100000,1.000000,19.000000,...,0.000000,6.000000,7.000000,5.000000,12.142857,0.000000,11.100000,1.400000,0.320000,NaN
75%,57387.750000,1.000000,39.000000,2.000000,9670.000000,1.000000,1.000000,140.000000,1.000000,37.000000,...,0.000000,6.000000,9.000000,6.000000,13.244048,0.000000,12.700000,2.600000,1.790000,NaN


In [8]:
train_data.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [9]:
train_data.dtypes

id                                                  int64
Marital status                                      int64
Application mode                                    int64
Application order                                   int64
Course                                              int64
Daytime/evening attendance                          int64
Previous qualification                              int64
Previous qualification (grade)                    float64
Nacionality                                         int64
Mother's qualification                              int64
Father's qualification                              int64
Mother's occupation                                 int64
Father's occupation                                 int64
Admission grade                                   float64
Displaced                                           int64
Educational special needs                           int64
Debtor                                              int64
Tuition fees u

In [10]:
train_data.isnull().any(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
76513    False
76514    False
76515    False
76516    False
76517    False
Length: 76518, dtype: bool

In [12]:
X, Y = train_data.drop(columns=['Target']), train_data['Target']
X.shape, Y.shape, train_data.shape

((76518, 37), (76518,), (76518, 38))

In [13]:
scaler = StandardScaler()
X_Scaled = scaler.fit_transform(X) # normalize data

In [14]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, random_state = 111, test_size = 0.20)
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

((61214, 37), (61214,), (15304, 37), (15304,))

### Bagger


In [14]:
bagger = BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=1.0, n_estimators=9, random_state=1111)
score = cross_val_score(bagger, X_Scaled, Y, cv=4)
print(f'Average validation accuracy is {np.mean(score)*100}%')

Average validation accuracy is 70.09047692726553%


### Random forest


In [15]:
randomF = RandomForestClassifier()
score = cross_val_score(randomF, X, Y, cv=4)
print(f'Average validation accuracy is {np.mean(score)*100}%')

Average validation accuracy is 81.44485433957801%


In [16]:
randomF = RandomForestClassifier()
score = cross_val_score(randomF, X_Scaled, Y, cv=4)
print(f'Average validation accuracy is {np.mean(score)*100}%')

Average validation accuracy is 81.68009966557975%


In [17]:
randomF100 = RandomForestClassifier(n_estimators=100, max_features="sqrt")
randomF100.fit(X_train,Y_train)
randomF100.score(X_val, Y_val)

0.8274960794563513

In [15]:
randomF1000 = RandomForestClassifier(n_estimators=1000, max_features="sqrt")
randomF1000.fit(X_train,Y_train)
randomF1000.score(X_val, Y_val)

0.8287375849451124

In [19]:
def submitModel(model, X_test):
    Y_tilda = model.predict(X_test)
    submission = pd.DataFrame(
        {'id': X_test["id"], 'Target': Y_tilda},
        columns = ['id', 'Target'])
    submission.to_csv('submission.csv', index = False)

In [21]:
submitModel(randomF1000, test_data) #0.82982
submitModel(randomF100, test_data) #0.82523

## Fidan


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [49]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

X_train = train_data.iloc[:, :-1].values  
y_train = train_data.iloc[:, -1].values    

# normalize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [52]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [82]:
model = keras.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)), 
    layers.Dense(50, activation='relu'),
    layers.Dense(100, activation='relu'), 
    layers.Dense(3, activation='softmax')  
])

In [83]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [84]:
X_train_scaled = np.array(X_train_scaled, dtype=np.float32)
history = model.fit(X_train_scaled, y_train_encoded, epochs=2, batch_size=32, validation_split=0.2)

Epoch 1/2
1913/1913 [==============================] - 6s 3ms/step - loss: 0.4966 - accuracy: 0.8056 - val_loss: 0.4725 - val_accuracy: 0.8150
Epoch 2/2
1913/1913 [==============================] - 5s 3ms/step - loss: 0.4640 - accuracy: 0.8215 - val_loss: 0.4691 - val_accuracy: 0.8177


In [85]:
X_test_scaled = scaler.transform(test_data)
predictions = model.predict(X_test_scaled)
predicted_indices = predictions.argmax(axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_indices)

  44/1595 [..............................] - ETA: 1s  

/gpfs/helios/home/juangonzalo/ML_project_UT/venv/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1595/1595 [==============================] - 2s 1ms/step


In [86]:
def submitModelPredictedLabels(predicted_labels, X_test):
    submission = pd.DataFrame(
        {'id': X_test["id"], 'Target': predicted_labels},
        columns = ['id', 'Target'])
    submission.to_csv('submission.csv', index = False)

In [87]:
submitModelPredictedLabels(predicted_labels, test_data)

In [88]:
# evaluate the model

In [89]:
assabler = pd.DataFrame()
X_test_scaled = scaler.transform(test_data)
assabler["model1"] = naiara_model.predict(X_test_scaled)
assabler["model2"] = randomF1000.predict(test_data)
assabler["model3"] = predicted_labels

/gpfs/helios/home/juangonzalo/ML_project_UT/venv/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [90]:
assabler['ensemble'] = assabler[["model1", "model2", "model3"]].mode(axis=1)[0]
print(assabler['ensemble'].head())
submitModelPredictedLabels(assabler['ensemble'], test_data)

0     Dropout
1    Graduate
2    Graduate
3    Graduate
4    Enrolled
Name: ensemble, dtype: object
